In [1]:
from carmon import registry_pipe
from carmon import common as cmc

Взглянем на ошибочные отработки

In [2]:
error_stage = registry_pipe.read_input_tables("test_data", 
                                              "test_data",
                                              separator="\t")

In [3]:
error_stage

{'success': False, 'payload': "[Errno 13] Permission denied: 'test_data'"}

In [4]:
error_stage = registry_pipe.read_input_tables("test_data/tab1.csv", 
                                              "test_data/tab2.csv",
                                              separator="\t")

In [5]:
error_stage

{'success': False,
 'payload': "[Errno 2] No such file or directory: 'test_data/tab1.csv'"}

Теперь проверим корректный запуск, передадим пути к файлам с таблицами

In [6]:
stage_1 = registry_pipe.read_input_tables("test_data/test_registry_pipe/FROM_2.txt", 
                                          "test_data/test_registry_pipe/FROM_3.txt",
                                          separator="\t")

Проверим успех операции

In [7]:
stage_1['success']

True

In [8]:
df = stage_1['payload']
df.head(3)

,litech_barcode,litech_sample_name,litech_region,litech_rna_pool,litech_registry_guess,barcode
64,90722,5793,Брянская область,123,641;624;643,barcode01_MN908947.3
65,90723,5701,Брянская область,123,641;624;643,barcode02_MN908947.3
67,90725,5610,Брянская область,123,641;624;643,barcode03_MN908947.3


Считываем таблицу со всеми реестрами

In [9]:
stage_2 = registry_pipe.read_all_registry_info("test_data/test_registry_pipe/registry.csv")

In [10]:
stage_2['success']

True

In [11]:
ref = stage_2['payload']
ref.head(3)

,registry_id,depart_name,sample_number,value
0,6537,ФБУЗ ЦГиЭ по Белгородской области,belg140SM160220,BE3050
1,6537,ФБУЗ ЦГиЭ по Белгородской области,belg140SM160222,BE3051
2,6537,ФБУЗ ЦГиЭ по Белгородской области,belg140SM160224,BE3052


Производим подготовительные манипуляции с таблицами

In [12]:
stage_3 = registry_pipe.append_desired_columns(df)

In [13]:
stage_3['success']

True

In [14]:
df = stage_3['payload']
df.head(3)

,barcode,litech_barcode,litech_sample_name,litech_region,region_short_name,registry_id,depart_name,sample_number,sample_name_value,registry_guess_status,valid_seq,sample_status_local,sample_vga_id,sample_status_remote,pango,nextclade,sequence_conclusion_local,sequence_vga_id,sequence_conclusion_remote
0,barcode01_MN908947.3,90722,5793,Брянская область,brya,,,,,,,,,,,,,,
1,barcode02_MN908947.3,90723,5701,Брянская область,brya,,,,,,,,,,,,,,
2,barcode03_MN908947.3,90725,5610,Брянская область,brya,,,,,,,,,,,,,,


Производим поиск образцов в реестре

In [15]:
stage_4 = registry_pipe.process_table_concatenation(df, ref)

In [16]:
stage_4['success']

True

In [17]:
stage_4['payload'].tail(3)  # тут конец таблицы, так как иначе видно личные данные образца

,barcode,litech_barcode,litech_sample_name,litech_region,region_short_name,registry_id,depart_name,sample_number,sample_name_value,registry_guess_status,valid_seq,sample_status_local,sample_vga_id,sample_status_remote,pango,nextclade,sequence_conclusion_local,sequence_vga_id,sequence_conclusion_remote
93,barcode94_MN908947.3,100033,393319,Рязанская область,ryaz,4517,ФБУЗ ЦГиЭ в Рязанской области,ryaz23SM75130,393319,OK,,,,,,,,,
94,barcode95_MN908947.3,100034,374619,Рязанская область,ryaz,4517,ФБУЗ ЦГиЭ в Рязанской области,ryaz23SM74866,374619,OK,,,,,,,,,
95,barcode96_MN908947.3,100035,359319,Рязанская область,ryaz,,,,,NO MATHCES,,,,,,,,,


In [18]:
optional_stage = cmc.save_concatenated_table(stage_4['payload'],
                                                       "test_data/test_registry_pipe/result.csv",
                                                       separator="\t")

In [19]:
optional_stage['success']

True

In [20]:
optional_stage['payload']

'Успешно сохранено в `test_data/test_registry_pipe/result.csv`'

Аналогично для сохранения есть возможность обработки ошибок. 

In [21]:
cmc.save_concatenated_table("nothing", "test_data/test_registry_pipe/result.csv", separator="\t")['payload']

"'str' object has no attribute 'to_csv'"

In [22]:
import pandas as pd
df = pd.read_csv("test_data/test_registry_pipe/result.csv", sep="\t", dtype=str, keep_default_na=False)
df.tail(3)

,barcode,litech_barcode,litech_sample_name,litech_region,region_short_name,registry_id,depart_name,sample_number,sample_name_value,registry_guess_status,valid_seq,sample_status_local,sample_vga_id,sample_status_remote,pango,nextclade,sequence_conclusion_local,sequence_vga_id,sequence_conclusion_remote
93,barcode94_MN908947.3,100033,393319,Рязанская область,ryaz,4517,ФБУЗ ЦГиЭ в Рязанской области,ryaz23SM75130,393319,OK,,,,,,,,,
94,barcode95_MN908947.3,100034,374619,Рязанская область,ryaz,4517,ФБУЗ ЦГиЭ в Рязанской области,ryaz23SM74866,374619,OK,,,,,,,,,
95,barcode96_MN908947.3,100035,359319,Рязанская область,ryaz,,,,,NO MATHCES,,,,,,,,,
